![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/15.Import_Transformers_Into_Spark_NLP.ipynb)

## Import Transformers from HuggingFace 🤗  into Spark NLP 🚀 
Let's keep in mind that this feature is only in Spark NLP 3.2.x and after. So please make sure you have upgraded to the latest Spark NLP release


## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.4.4` version and Transformers on `4.15.0`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q transformers==4.15.0 tensorflow==2.4.4

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [dslim/bert-base-NER](https://huggingface.co/dslim/bert-base-NER) model from HuggingFace as an example
- In addition to `TFBertForTokenClassification` we also need to save the `BertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFBertForTokenClassification, BertTokenizer 

MODEL_NAME = 'dslim/bert-base-NER'

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFBertForTokenClassification.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFBertForTokenClassification.from_pretrained(MODEL_NAME, from_pt=True)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

try downloading TF weights


Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForTokenClassification.

All the layers of TFBertForTokenClassification were initialized from the model checkpoint at dslim/bert-base-NER.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 421080
-rw-r--r-- 1 root root       999 Oct  5 13:25 config.json
drwxr-xr-x 3 root root      4096 Oct  5 13:25 saved_model
-rw-r--r-- 1 root root 431174752 Oct  5 13:25 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 7896
drwxr-xr-x 2 root root    4096 Oct  5 13:25 assets
-rw-r--r-- 1 root root 8073862 Oct  5 13:25 saved_model.pb
drwxr-xr-x 2 root root    4096 Oct  5 13:25 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 220
-rw-r--r-- 1 root root    112 Oct  5 13:24 special_tokens_map.json
-rw-r--r-- 1 root root    552 Oct  5 13:24 tokenizer_config.json
-rw-r--r-- 1 root root 213450 Oct  5 13:24 vocab.txt


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `vocab.txt` from the tokenizer
- All we need is to just copy the `vocab.txt` to `saved_model/1/assets` which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [ ]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

!cp {MODEL_NAME}_tokenizer/vocab.txt {asset_path}

In [ ]:
# get label2id dictionary 
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [ ]:
! ls -l {MODEL_NAME}/saved_model/1/assets

total 216
-rw-r--r-- 1 root root     51 Oct  5 13:25 labels.txt
-rw-r--r-- 1 root root 213450 Oct  5 13:25 vocab.txt


## Import and Save BertForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab

In [ ]:
! pip install -q pyspark==3.3.0 spark-nlp==4.2.4

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp

spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `BertForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BertForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` only accepts local paths and not distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. That is why we use `write.save` so we can use `.load()` from any file systems



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

tokenClassifier = BertForTokenClassification.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
! rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome!

This is your BertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 428476
-rw-r--r-- 1 root root 438746116 Oct  5 13:27 bert_classification_tensorflow
drwxr-xr-x 5 root root      4096 Oct  5 13:27 fields
drwxr-xr-x 2 root root      4096 Oct  5 13:27 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForTokenClassification model 

In [ ]:
tokenClassifier_loaded = BertForTokenClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

That's it! You can now go wild and use hundreds of `BertForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀 


You can see what labels were used to train this model via `getClasses` function:

In [ ]:
tokenClassifier_loaded.getClasses()

['B-LOC', 'I-ORG', 'I-MISC', 'I-LOC', 'I-PER', 'B-MISC', 'B-ORG', 'O', 'B-PER']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded    
])

# couple of simple examples
example = spark.createDataFrame([["My name is Sarah and I live in London"],
                                 ["My name is Clara and I live in Berkeley, California."]]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "ner.result").show(truncate=False)

+----------------------------------------------------+------------------------------------------------+
|text                                                |result                                          |
+----------------------------------------------------+------------------------------------------------+
|My name is Sarah and I live in London               |[O, O, O, B-PER, O, O, O, O, B-LOC]             |
|My name is Clara and I live in Berkeley, California.|[O, O, O, B-PER, O, O, O, O, B-LOC, O, B-LOC, O]|
+----------------------------------------------------+------------------------------------------------+

